# Reinforcement Learning Lab: Tic-Tac-Toe with Pygame Interface

<img src="../images/tictactoe.png" alt="Tic Tac Toe Example" width="200"/>

**What is this lab about?**  

In this lab, we’ll create a **graphical interface** for Tic-Tac-Toe using **Pygame**. The interface will allow:

- A human to play against a trained **RL agent** (SARSA, Q-Learning, or DQN).  
- Students to understand **how to design and integrate a user interface** with a machine learning model.  
- Visualization of **game states, moves, and outcomes**.  

We will focus on:  

- Understanding the **event-driven programming model** in Pygame.  
- Designing a **clickable Tic-Tac-Toe board**.  
- Integrating a **trained agent** for automatic moves.  
- Handling **game states, wins, draws, and illegal moves**.  
- Providing **interactive feedback** to the user.  



## Table of Contents

- [1 - Packages](#1)  
- [2 - Initialize Pygame and Board](#2)  
- [3 - Draw Tic-Tac-Toe Board](#3)  
- [4 - Handling Player Moves](#4)  
- [5 - Integrate RL Agent](#5)  
- [6 - Game Loop and Display](#6)  
- [7 - Optional Features](#7)  
- [8 - Exercises](#8) 

## 1 - Packages <a id="1"></a>

**Instructions:**  

- Import the necessary Python packages.  
- Ensure `pygame`, `numpy`, and `torch` are installed for graphical interface and agent integration.

In [1]:
import pygame
import numpy as np
import torch
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.12.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


## 2 - Initialize Pygame and Board <a id="2"></a>

**Instructions:**  

- Initialize the Pygame window.  
- Set screen size, colors, and caption.  
- Initialize a 3x3 board using a NumPy array where:  
  - `0` = empty cell  
  - `1` = human player  
  - `-1` = RL agent  

> Students should understand that the **board state** is stored separately from the display, and each update in the game loop will read from this array to draw X and O.


In [2]:
# Initialize Pygame
pygame.init()

# Screen settings
size = width, height = 300, 300
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Tic-Tac-Toe vs RL Agent")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)

# Initialize board: 0-empty, 1-player, -1-agent
board = np.zeros((3,3), dtype=int)


## 3 - Draw Tic-Tac-Toe Board <a id="3"></a>

**Instructions:**  

- Draw a **3x3 grid** on the Pygame window.  
- Use **lines** to separate rows and columns.  
- Draw **X** for the human player and **O** for the RL agent according to the board state.  
- Update the display after each change.  

> Hint: should understand how the **board state array** maps to screen coordinates and how to render X and O in the correct cells.


In [4]:
def draw_board(board):
    # Fill background
    screen.fill(WHITE)
    
    # Draw grid lines
    for i in range(1, 3):
        pygame.draw.line(screen, BLACK, (0, i*100), (300, i*100), 3)  # horizontal
        pygame.draw.line(screen, BLACK, (i*100, 0), (i*100, 300), 3)  # vertical
    
    # Draw X and O based on board state
    for i in range(3):
        for j in range(3):
            if board[i,j] == 1:  # Player X
                pygame.draw.line(screen, BLUE, (j*100+10,i*100+10), (j*100+90,i*100+90), 5)
                pygame.draw.line(screen, BLUE, (j*100+90,i*100+10), (j*100+10,i*100+90), 5)
            elif board[i,j] == -1:  # Agent O
                pygame.draw.circle(screen, RED, (j*100+50,i*100+50), 40, 5)
    
    # Update display
    pygame.display.update()


## 4 - Handling Player Moves <a id="4"></a>

**Instructions:**  

- Map **mouse clicks** to board positions.  
- Only allow moves in **empty cells**.  
- Update the board array to reflect the human player’s move (`1`).  
- Students should understand **mapping pixel coordinates to board indices**.  
- This is the first step toward making the board interactive.  


In [2]:
def player_move(board, mouse_pos):
    """
    Update the board with a player move based on mouse click.
    
    Parameters:
        board (np.array): Current game board (3x3)
        mouse_pos (tuple): (x, y) position of the mouse click
    Returns:
        bool: True if move was valid, False otherwise
    """
    row, col = mouse_pos[1] // 100, mouse_pos[0] // 100  # Map pixels to board index
    if board[row, col] == 0:
        board[row, col] = 1  # Player move
        return True
    return False


In [3]:
running = True
while running:
    draw_board(board)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if player_move(board, event.pos):
                print("Player moved!")


NameError: name 'draw_board' is not defined

## 5 - Integrate RL Agent <a id="5"></a>

**Instructions:**  

- Use a trained RL model (DQN, Q-Learning, or SARSA) to select moves.  
- Ensure the agent only selects **empty cells**.  
- Update the board array with the agent’s move (`-1`).  
- Students should understand:  
  - Flattening the board state for the model input.  
  - Mapping model output (0–8) back to **row and column**.  
  - Handling cases where the predicted cell is already occupied.  


In [4]:
def agent_move(board, model):
    """
    Update the board with the agent's move using the trained model.
    
    Parameters:
        board (np.array): Current game board (3x3)
        model (torch.nn.Module): Trained RL model predicting Q-values
    """
    # Flatten the board for model input
    state = board.flatten()
    
    with torch.no_grad():
        q_values = model(torch.FloatTensor(state))
        action = int(torch.argmax(q_values).item())
    
    # If chosen cell is occupied, select a random empty cell
    if board[action // 3, action % 3] != 0:
        empty_cells = [i for i in range(9) if board.flatten()[i] == 0]
        if empty_cells:
            action = random.choice(empty_cells)
    
    # Update the board
    board[action // 3, action % 3] = -1


In [ ]:
# Example: simulate a player click at (50, 50)
player_move(board, (50, 50))
draw_board(board)

# Agent makes a move
agent_move(board, model)  # 'model' should be a trained PyTorch model
draw_board(board)


## 6 - Game Loop and Display <a id="6"></a>

**Instructions:**  

- Combine **player clicks**, **agent moves**, and **drawing functions** in a single game loop.  
- Check for **win, draw, or ongoing game** after each move.  
- Update the display after every change.  
- Students should understand **event-driven programming** and how the loop continuously checks for input and updates the UI.


In [5]:
# Initialize game state
running = True
game_over = False

def check_winner(board):
    """
    Returns:
        1 if player wins
       -1 if agent wins
        0 if draw
        None if game ongoing
    """
    # Rows, columns, diagonals
    lines = list(board) + list(board.T) + [board.diagonal(), np.fliplr(board).diagonal()]
    for line in lines:
        if np.all(line == 1):
            return 1
        elif np.all(line == -1):
            return -1
    if not np.any(board == 0):
        return 0  # Draw
    return None  # Game ongoing

# Game loop
while running:
    draw_board(board)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            if player_move(board, event.pos):
                # Check for win/draw after player move
                result = check_winner(board)
                if result is not None:
                    game_over = True
                    print("Game result:", "Player wins!" if result==1 else "Agent wins!" if result==-1 else "Draw!")
                    continue
                
                # Agent move
                agent_move(board, model)
                
                # Check for win/draw after agent move
                result = check_winner(board)
                if result is not None:
                    game_over = True
                    print("Game result:", "Player wins!" if result==1 else "Agent wins!" if result==-1 else "Draw!")
                    
pygame.quit()


NameError: name 'draw_board' is not defined

## 7 - Optional Features <a id="7"></a>

Students can enhance the Pygame Tic-Tac-Toe game with these optional features:

- **Highlight the winning line**: draw a colored line over the row, column, or diagonal that wins.  
- **Restart button**: allow the player to reset the board without closing the window.  
- **Choose X or O**: let the human player select their symbol at the start.  
- **Scoreboard**: display wins, draws, and losses across multiple games.  
- **Animations or sounds**: add visual effects or sounds for moves and wins.  
- **Multiple RL agents**: allow the player to choose between SARSA, Q-Learning, or DQN agents.  
- **Different board sizes**: experiment with 4x4 or 5x5 grids and adapt the agent accordingly.  

> Students should be encouraged to **think creatively** about improving the interface and adding features.


## 8 - Exercises <a id="8"></a>

1. **Add a scoreboard**: track wins, draws, and losses for the player and agent.  
2. **Adaptive agent**: let the agent continue learning from the player’s moves in real time.  
3. **Menu for agent selection**: choose which RL model to play against (SARSA, Q-Learning, DQN).  
4. **Move animations**: animate the drawing of X and O instead of instant placement.  
5. **Variable board size**: implement 4x4 or 5x5 boards and adapt the agent logic.  
6. **Undo move feature**: allow the player to undo their last move.  
7. **Difficulty levels**: let the agent play at different levels (random, heuristic, fully trained).  

> These exercises encourage students to **integrate UI design, game logic, and RL model interaction**, reinforcing the project-based learning approach.
